In [1]:
import pandas as pd
from scipy.spatial.distance import pdist
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.pipeline import Pipeline
from skbio.stats.ordination import pcoa
from skbio import TreeNode
from q2_anomaly_detection.datasets import KeyboardDataset
from q2_anomaly_detection.transforms import Rarefaction, CLR, AsDense, UniFrac, RarefactionBIOM
from q2_anomaly_detection.benchmark import Benchmark, ColumnValueSplitter, ExternalScorer, Scorer
from q2_anomaly_detection.utils import as_dense, TrivialScorer
from q2_anomaly_detection.cross_validation import column_value_splitter
%matplotlib inline

In [2]:
RANDOM_SEED = 724
RAREFACTION_DEPTH = 500
# qiita study 232 with trimming to 90 bp and pick closed reference on 97 otus tree
dataset = KeyboardDataset('data/keyboard')

Files already downloaded and verified


In [3]:
TRAINING_CATEGORY = 'host_subject_id_for_surface'
TRUTH_CATEGORY = 'host_subject_id'

In [4]:
def construct_category(metadata):
    metadata['host_subject_id_for_surface'] = np.nan
    skin_samples = metadata.index[metadata.sample_type == 'skin']
    metadata.loc[:, 'host_subject_id_for_surface'].loc[skin_samples] = metadata.host_subject_id[
        skin_samples].to_numpy()


def subset_metadata(metadata_all):
    subject_ids = ['M2','M3', 'M9']
    metadata = metadata_all.query('host_subject_id in @subject_ids')
    return metadata

In [5]:
table = dataset['table']

In [6]:
metadata_all = dataset['metadata']
metadata = subset_metadata(metadata_all)
metadata = metadata.set_index('sample_name')

In [7]:
construct_category(metadata)
table.filter(metadata.index, axis='sample')

/home/kalen/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


2180 x 104 <class 'biom.table.Table'> with 12380 nonzero entries (5% dense)

In [8]:
np.random.seed(RANDOM_SEED)
subtable = table.subsample(RAREFACTION_DEPTH)
submetadata = metadata.loc[metadata.index.isin(subtable.ids('sample'))]

In [9]:
study_ids = subtable.ids('sample')

In [10]:
baseline = Pipeline(steps=[
    ('asdense', AsDense()),
    ('baseline', TrivialScorer()),
])
iso = Pipeline(steps=[
    ('asdense', AsDense()),
    ('rarefy', Rarefaction(RAREFACTION_DEPTH,
                           replace=True,
                           )),
    ('IF', IsolationForest(contamination="auto")),
])
lof = Pipeline(steps=[
    ('asdense', AsDense()),
    ('rarefy', Rarefaction(RAREFACTION_DEPTH,
                           replace=True,
                           )),
    ('LOF', LocalOutlierFactor(
        novelty=True,
        contamination="auto",
        )
     ),
])
lof_jaccard = Pipeline(steps=[
    ('asdense', AsDense()),
    ('rarefy', Rarefaction(RAREFACTION_DEPTH,
                           replace=True,
                           )),
    ('LOF', LocalOutlierFactor(
        novelty=True,
        contamination="auto",
        metric='jaccard',
        )
     ),
])
ocsvm = Pipeline(steps=[
    ('asdense', AsDense()),
    ('rarefy', Rarefaction(RAREFACTION_DEPTH,
                           replace=True,
                           )),
    ('SVM', OneClassSVM(gamma='auto')
     ),
])

iso_clr = Pipeline(steps=[
    ('asdense', AsDense()),
    ('clr', CLR()),
    ('IF', IsolationForest(contamination="auto")),
])
lof_clr = Pipeline(steps=[
    ('asdense', AsDense()),
    ('clr', CLR()),
    ('LOF', LocalOutlierFactor(
        novelty=True,
        contamination="auto",
        )
     ),
])
lof_jaccard_clr = Pipeline(steps=[
    ('asdense', AsDense()),
    ('clr', CLR()),
    ('LOF', LocalOutlierFactor(
        novelty=True,
        contamination="auto",
        metric='jaccard',
        )
     ),
])
ocsvm_clr = Pipeline(steps=[
    ('asdense', AsDense()),
    ('clr', CLR()),
    ('SVM', OneClassSVM(gamma='auto')
     ),
])

# CONVENTION: anomaly scores should be transformed onto (0, 1) where 1 is
#  least anomalous and 0 is the most anomalous
models = {
    'baseline': {
        'model': baseline,
    },
    'IF': {
        'model': iso,
    },
    'LOF': {
        'model': lof,
    },
    'LOF-jaccard': {
        'model': lof_jaccard,
    },
    'ocsvm': {
        'model': ocsvm,
    },
    'IF-clr': {
        'model': iso_clr,
    },
    'LOF-clr': {
        'model': lof_clr,
    },
    'LOF-jaccard-clr': {
        'model': lof_jaccard_clr,
    },
    'ocsvm-clr': {
        'model': ocsvm_clr,
    },
}

In [58]:
tree = tree_path = 'data/resources/97_otus.tree'
class PseudoTimeSplitter:
    def __init__(self, training_category, truth_category):
        self.training_category = training_category
        self.truth_category = truth_category
        
    def split(self, table, metadata):
        """
        Yields
        ------
        label : str
            Name for the split
        training_table : BIOMTable
            Data for fitting detection model
        train_ids : set(str)
            A collection of train ids
        test_table : BIOMTable
            Data for testing
        test_labels : arrary like of shape (n_samples,)
            Values should be 0 or 1. 1 is anomaly and 0 is not
        """
        # returns generator
        col_val_split = column_value_splitter(
            table, metadata, self.training_category,
        )
        dm_pipeline = Pipeline(
                [
                    ('rarefy', RarefactionBIOM(RAREFACTION_DEPTH)),
                    ('unifrac', UniFrac(tree_path)),
                ]
            )
        for label, ids, training_table in col_val_split:
            # testing
            # removing training samples from the table with all data
            dm = dm_pipeline.fit_transform(training_table)

            pcoa_results = pcoa(dm)
            ordination = pcoa_results.samples
            sample_order = ordination["PC1"].sort_values().index.values
            sample_order = [int(i) for i in sample_order]
            sample_ids_ordered = training_table.ids("sample")[sample_order]
            
            training_table = training_table.sort_order(sample_ids_ordered)
            train_ids = set(sample_ids_ordered)
            test_labels = np.zeros(len(train_ids))
            test_labels[0] = 1
            yield label, training_table, train_ids, None, test_labels
            
class PseudoTimeScorer(Scorer):
    
    def score(self, context):
        model = context["model"]      
        return model.scores_
    
    def add_scores():
        pass

class MockTimeSeriesModel:
    
    def fit(self, X, y=None):
        self.scores_ = np.arange(len(X.ids("sample")))
        return self
        

In [59]:
splitter = PseudoTimeSplitter(TRAINING_CATEGORY, TRUTH_CATEGORY)
external_scorer = PseudoTimeScorer()
models = {
    "mock": {
        "model": MockTimeSeriesModel()
    }
}
benchmark = Benchmark(models)
benchmark.set_scorer(external_scorer)
benchmark.set_splitter(splitter)
all_results = benchmark.benchmarking_loop(subtable, submetadata)

AttributeError: 'NoneType' object has no attribute 'ids'

In [16]:
all_results.short_form()

,model_name,category,roc_auc,avg_prec
0,baseline,M2,0.500000,0.685393
1,baseline,M3,0.500000,0.752809
2,baseline,M9,0.500000,0.786517
3,IF,M2,0.006440,0.475381
4,IF,M3,0.725916,0.913407
5,IF,M9,0.896241,0.967330
6,LOF,M2,1.000000,1.000000
7,LOF,M3,0.805970,0.907395
8,LOF,M9,0.662406,0.864965
9,LOF-jaccard,M2,0.995902,0.998263
